In [1]:
import os
import tempfile

# import scanpy as sc
import scvi
# import pyreadr
# import seaborn as sns
# import torch
# from rich import print
# from scib_metrics.benchmark import Benchmarker

/mnt/sda4/conda/envs/scvi/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/mnt/sda4/conda/envs/scvi/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/mnt/sda4/conda/envs/scvi/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/mnt/sda4/conda/envs/scvi/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing CSCDataset from `anndata.experimental` is deprecated. Import anndata.abc.CSCDataset instead.
  warnings.warn(msg, FutureWarning)
/mnt/sda4/conda/envs/scvi/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing CSRDataset from `anndata

In [2]:
adata = scvi.data.read_h5ad('/mnt/sda4/singleCell_LAB/HitGlio_sample1-test/RDS/filename.h5ad')

In [3]:
adata

AnnData object with n_obs × n_vars = 36553 × 51448
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO', 'nFeature_HTO', 'nCount_ADT', 'nFeature_ADT', 'MULTI_ID', 'MULTI_classification', 'percent_ribo', 'percent_mito', 'percent_hemoglob', 'log10GenesPerUmi', 'RNA_snn_res.0.5', 'seurat_clusters', 'x', 'y', 'ident', 'cluster', 'cell', 'sample'
    var: 'vf_mvp_data.1_mvp.mean', 'vf_mvp_data.1_mvp.dispersion', 'vf_mvp_data.1_mvp.dispersion.scaled', 'vf_mvp_data.1_variable', 'vf_mvp_data.1_rank', 'vf_vst_counts.1_mean', 'vf_vst_counts.1_variance', 'vf_vst_counts.1_variance.expected', 'vf_vst_counts.1_variance.standardized', 'vf_vst_counts.1_variable', 'vf_vst_counts.1_rank', 'vf_vst_counts.2_mean', 'vf_vst_counts.2_variance', 'vf_vst_counts.2_variance.expected', 'vf_vst_counts.2_variance.standardized', 'vf_vst_counts.2_variable', 'vf_vst_counts.2_rank', 'var.features', 'var.features.rank'
    obsm: 'X_pca'

In [ ]:
scvi.settings.num_threads = 46
scvi.settings.dl_num_workers = 46
# run setup_anndata, use column stim for batch
# scvi.model.SCVI.setup_anndata(adata, batch_key = 'MULTI_ID')

# create the model
model = scvi.model.SCVI(adata)

# train t.e model
model.train()

/mnt/sda4/conda/envs/scvi/lib/python3.12/site-packages/scvi/data/fields/_layer_field.py:115: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  _verify_and_correct_data_format(adata, self.attr_name, self.attr_key)
/mnt/sda4/conda/envs/scvi/lib/python3.12/site-packages/scvi/dataloaders/_data_splitting.py:210: UserWarning: Last batch will have a small size of 2samples. Consider changing settings.batch_size or batch_size in model.traincurrently 128 to avoid errors during model training.
  self.n_train, self.n_val = validate_data_split(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Epoch 1/219:   0%|                               | 0/219 [15:40<?, ?it/s]
Epoch 37/219:  16%|▏| 36/219 [1:07:08<5:25:34, 106.74s/it, v_num=1, train

In [7]:
model

SCVI model with the following parameters: 
n_hidden: 128, n_latent: 10, n_layers: 1, dropout_rate: 0.1, dispersion: gene, gene_likelihood: zinb, 
latent_distribution: normal.
Training status: Trained
Model's adata is minified?: False

In [9]:
latent = model.get_latent_representation()

In [10]:
latent

array([[ 0.59444225, -0.79490936, -0.41424984, ...,  1.7820519 ,
         0.10756576,  1.3241537 ],
       [ 1.2542677 , -0.94397736, -0.73620677, ...,  0.12226555,
         1.0367217 ,  1.0107079 ],
       [-2.5456603 , -3.3137777 , -0.03678724, ...,  3.242712  ,
        -2.7642903 ,  1.4490888 ],
       ...,
       [-0.26232213,  0.7610818 ,  0.07071286, ...,  1.5541439 ,
         0.49503773, -0.21972764],
       [-0.39786083,  1.2939185 ,  0.2314294 , ...,  1.1710755 ,
         0.17902517, -0.38895202],
       [ 0.07945365,  1.3293488 , -0.1164563 , ...,  1.647945  ,
        -0.06979358, -0.6976447 ]], dtype=float32)

In [22]:
import pandas as pd 
import numpy as np

In [12]:
latent_df = pd.DataFrame(latent)

In [13]:
latent_df.to_csv("latent.csv")

In [ ]:
# below I try to model rna and adt data together, (i think it will help with labeling clusters, but not sure)

In [64]:
import mudata as md
import scanpy as sc
import scvi
import pandas as pd 
import numpy as np

In [65]:
scvi.settings.num_threads = 46
scvi.settings.dl_num_workers = 46

In [66]:
adata_rna = scvi.data.read_h5ad('/mnt/sda4/singleCell_LAB/HitGlio_sample1-test/RDS/seu_singlet_sct-doublet-500-hvg_05dec24.rds.h5ad')

In [67]:
adata_rna

AnnData object with n_obs × n_vars = 13409 × 25571
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO', 'nFeature_HTO', 'nCount_ADT', 'nFeature_ADT', 'MULTI_ID', 'MULTI_classification', 'percent_ribo', 'percent_mito', 'percent_hemoglob', 'log10GenesPerUmi', 'S.Score', 'G2M.Score', 'Phase', 'old.ident', 'nCount_SCT', 'nFeature_SCT', 'cluster.sct', 'seurat_clusters', 'pANN_0.25_0.09_1841', 'DF.classifications_0.25_0.09_1841'
    var: 'name'
    obsm: 'X_pca.sct', 'X_umap.sct'

In [68]:
# adata_rna.layers["counts"] = adata.X.copy().todense() /only adt needs to be dense

adata_rna.layers["counts"] = adata_rna.X.copy()

In [69]:
adata_rna.layers

Layers with keys: counts

In [70]:
adata_adt = scvi.data.read_h5ad('/mnt/sda4/singleCell_LAB/HitGlio_sample1-test/RDS/seu_singlet_adt-doublet-500-hvg_05dec24.rds.h5ad')

In [71]:
adata_adt.X = adata_adt.X.todense()

/mnt/sda4/conda/envs/scvi/lib/python3.12/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)


In [72]:
adata_adt.obs_names_make_unique()

In [73]:
mdata = md.MuData({"rna": adata_rna, "protein": adata_adt})

/mnt/sda4/conda/envs/scvi/lib/python3.12/site-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/mnt/sda4/conda/envs/scvi/lib/python3.12/site-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


In [74]:
mdata

MuData object with n_obs × n_vars = 13409 × 25613
  var:	'name'
  2 modalities
    rna:	13409 x 25571
      obs:	'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO', 'nFeature_HTO', 'nCount_ADT', 'nFeature_ADT', 'MULTI_ID', 'MULTI_classification', 'percent_ribo', 'percent_mito', 'percent_hemoglob', 'log10GenesPerUmi', 'S.Score', 'G2M.Score', 'Phase', 'old.ident', 'nCount_SCT', 'nFeature_SCT', 'cluster.sct', 'seurat_clusters', 'pANN_0.25_0.09_1841', 'DF.classifications_0.25_0.09_1841'
      var:	'name'
      obsm:	'X_pca.sct', 'X_umap.sct'
      layers:	'counts'
    protein:	13409 x 42
      obs:	'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO', 'nFeature_HTO', 'nCount_ADT', 'nFeature_ADT', 'MULTI_ID', 'MULTI_classification', 'percent_ribo', 'percent_mito', 'percent_hemoglob', 'log10GenesPerUmi', 'S.Score', 'G2M.Score', 'Phase', 'old.ident', 'nCount_SCT', 'nFeature_SCT', 'cluster.sct', 'seurat_clusters', 'pANN_0.25_0.09_1841', 'DF.classifications_0.25_0.09_1841'
      var:	'name'
      obsm:	'X_pca.sct', 'X_umap.sct'

In [55]:
# mdata["rna"].var["vf_vst_counts_variable"]


AAACCAAAGACCTACA-1    False
AAACCAAAGACTGATT-1    False
AAACCAAAGAGAATCA-1    False
AAACCAAAGCCATTGA-1    False
AAACCAAAGGTCTGGT-1    False
                      ...  
GTTGTGGGTATGATGC-1    False
GTTGTGGGTCCTATGC-1    False
GTTGTGGGTGCATTCC-1    False
GTTGTGGGTGGCGATC-1    False
GTTGTGGGTGGGTTTG-1    False
Name: sct.var.genes, Length: 13409, dtype: bool

In [57]:
# mdata.mod["rna_subset"] = mdata.mod["rna"][:, mdata.mod["rna"].var["vf_vst_counts_variable"]].copy()


IndexError: Boolean index does not match AnnData’s shape along this dimension. Boolean index has shape (13409,) while AnnData index has shape (25571,).

In [75]:
mdata.update()

/mnt/sda4/conda/envs/scvi/lib/python3.12/site-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/mnt/sda4/conda/envs/scvi/lib/python3.12/site-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


/mnt/sda4/conda/envs/scvi/lib/python3.12/site-packages/scvi/data/fields/_layer_field.py:115: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  _verify_and_correct_data_format(adata, self.attr_name, self.attr_key)


In [76]:
# scvi.model.TOTALVI.setup_mudata(
#     mdata,
#     rna_layer="counts",
#     protein_layer=None,
#     modalities={
#         "rna_layer": "rna_subset",
#         "protein_layer": "protein",
#         "batch_key": "rna_subset",
#     },
# )

scvi.model.TOTALVI.setup_mudata(
    mdata,
    rna_layer="counts",
    # batch_key = "MULTI_ID",
    protein_layer=None,
    modalities={
        "rna_layer": "rna",
        "protein_layer": "protein",
        "batch_key": "rna",
    },
)


/mnt/sda4/conda/envs/scvi/lib/python3.12/site-packages/scvi/data/fields/_layer_field.py:115: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  _verify_and_correct_data_format(adata, self.attr_name, self.attr_key)


In [78]:
model3 = scvi.model.TOTALVI(mdata)

INFO     Computing empirical prior initialization for protein background.                                          


In [79]:
model3.train()

latent = model3.get_latent_representation()

latent_df = pd.DataFrame(latent)
latent_df.to_csv("latent-sct-totalVI_06dec24_wo-int-doublet-500.csv")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/mnt/sda4/conda/envs/scvi/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/mnt/sda4/conda/envs/scvi/lib/python3.12/site-packages/lightning/pytorch/core/optimizer.py:316: The lr scheduler dict contains the key(s) ['monitor'], but the keys will be ignored. You need to call `lr_scheduler.step()` manually in manual optimization.


Epoch 217/400:  54%|▌| 217/400 [2:04:27<1:44:57, 34.41s/it, v_num=1, tr
Monitored metric elbo_validation did not improve in the last 45 records. Best score: 7391.365. Signaling Trainer to stop.
